In [1]:
import math
import torch
import gpytorch

ModuleNotFoundError: No module named 'torch'

# Dataset

$ y = \sin(2\pi x) + \epsilon $

$ x \in [0,1] $

$ \epsilon \sim \mathcal{N}(0, 0.04)$

In [25]:
n = 2**13

X_train = torch.linspace(0,1,n)
y_train = torch.sin(X_train*(2*math.pi)) + torch.randn(X_train.size())*math.sqrt(0.04)

# GP Regression CPU

https://docs.gpytorch.ai/en/stable/examples/01_Exact_GPs/Simple_GP_Regression.html

In [26]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, X_train, y_train, likelihood):
        super(ExactGPModel, self).__init__(X_train, y_train, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [27]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(X_train, y_train, likelihood)

In [28]:
model.train()
likelihood.train()

GaussianLikelihood(
  (noise_covar): HomoskedasticNoise(
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
)

In [29]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

In [30]:
n_iters = 20

for i in range(n_iters):
    optimizer.zero_grad()
    output = model(X_train)
    loss = -mll(output, y_train)
    loss.backward()
    print("Iter %d/%d - Loss: %.3f \t lengthscale: %.3f \t noise: %.3f" % 
          (
              i+1,
              n_iters,
              loss.item(),
              model.covar_module.base_kernel.lengthscale.item(),
              model.likelihood.noise.item()
          )
    )
    optimizer.step()

Iter 1/20 - Loss: 0.795 	 lengthscale: 0.693 	 noise: 0.693
Iter 2/20 - Loss: 0.752 	 lengthscale: 0.644 	 noise: 0.644
Iter 3/20 - Loss: 0.710 	 lengthscale: 0.599 	 noise: 0.598
Iter 4/20 - Loss: 0.671 	 lengthscale: 0.557 	 noise: 0.554
Iter 5/20 - Loss: 0.632 	 lengthscale: 0.519 	 noise: 0.513
Iter 6/20 - Loss: 0.594 	 lengthscale: 0.486 	 noise: 0.474
Iter 7/20 - Loss: 0.556 	 lengthscale: 0.456 	 noise: 0.437
Iter 8/20 - Loss: 0.518 	 lengthscale: 0.430 	 noise: 0.403
Iter 9/20 - Loss: 0.480 	 lengthscale: 0.407 	 noise: 0.370
Iter 10/20 - Loss: 0.442 	 lengthscale: 0.387 	 noise: 0.340
Iter 11/20 - Loss: 0.404 	 lengthscale: 0.369 	 noise: 0.312
Iter 12/20 - Loss: 0.367 	 lengthscale: 0.354 	 noise: 0.286
Iter 13/20 - Loss: 0.329 	 lengthscale: 0.341 	 noise: 0.262
Iter 14/20 - Loss: 0.292 	 lengthscale: 0.329 	 noise: 0.240
Iter 15/20 - Loss: 0.254 	 lengthscale: 0.319 	 noise: 0.219
Iter 16/20 - Loss: 0.218 	 lengthscale: 0.309 	 noise: 0.200
Iter 17/20 - Loss: 0.182 	 length

/home/c3/.conda/envs/py-gordon-ML_2_0_0/lib/python3.8/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(


Iter 20/20 - Loss: 0.078 	 lengthscale: 0.283 	 noise: 0.138


# Using the GPU

https://docs.gpytorch.ai/en/stable/examples/02_Scalable_Exact_GPs/Simple_GP_Regression_CUDA.html

In [31]:
# re-define the model and send it to GPU
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(X_train, y_train, likelihood)
X_train_gpu = X_train.cuda()
y_train_gpu = y_train.cuda()
model_gpu = model.cuda()
likelihood_gpu = likelihood.cuda()

In [32]:
model_gpu.train()
likelihood_gpu.train()

GaussianLikelihood(
  (noise_covar): HomoskedasticNoise(
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
)

In [33]:
optimizer_gpu = torch.optim.Adam(model_gpu.parameters(), lr=0.1)
mll_gpu = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood_gpu, model_gpu)

In [34]:
n_iters = 20

for i in range(n_iters):
    optimizer_gpu.zero_grad()
    output = model_gpu(X_train_gpu)
    loss = -mll_gpu(output, y_train_gpu)
    loss.backward()
    print("Iter %d/%d - Loss: %.3f \t lengthscale: %.3f \t noise: %.3f" % 
          (
              i+1,
              n_iters,
              loss.item(),
              model_gpu.covar_module.base_kernel.lengthscale.item(),
              model_gpu.likelihood.noise.item()
          )
    )
    optimizer_gpu.step()

Iter 1/20 - Loss: 0.795 	 lengthscale: 0.693 	 noise: 0.693
Iter 2/20 - Loss: 0.752 	 lengthscale: 0.644 	 noise: 0.644
Iter 3/20 - Loss: 0.710 	 lengthscale: 0.599 	 noise: 0.598
Iter 4/20 - Loss: 0.671 	 lengthscale: 0.557 	 noise: 0.554
Iter 5/20 - Loss: 0.632 	 lengthscale: 0.519 	 noise: 0.513
Iter 6/20 - Loss: 0.594 	 lengthscale: 0.486 	 noise: 0.474
Iter 7/20 - Loss: 0.556 	 lengthscale: 0.456 	 noise: 0.437
Iter 8/20 - Loss: 0.518 	 lengthscale: 0.430 	 noise: 0.403
Iter 9/20 - Loss: 0.480 	 lengthscale: 0.407 	 noise: 0.370
Iter 10/20 - Loss: 0.442 	 lengthscale: 0.387 	 noise: 0.340
Iter 11/20 - Loss: 0.404 	 lengthscale: 0.369 	 noise: 0.312
Iter 12/20 - Loss: 0.367 	 lengthscale: 0.354 	 noise: 0.286
Iter 13/20 - Loss: 0.329 	 lengthscale: 0.341 	 noise: 0.262
Iter 14/20 - Loss: 0.292 	 lengthscale: 0.329 	 noise: 0.240
Iter 15/20 - Loss: 0.254 	 lengthscale: 0.319 	 noise: 0.219
Iter 16/20 - Loss: 0.218 	 lengthscale: 0.309 	 noise: 0.200
Iter 17/20 - Loss: 0.182 	 length